# Problem Formulation

Objective: maximize the discount utility from consumption

$$
\underset{c_1,c_2,\cdots ,c_T}{\max}\sum_{i=1}^T{\beta ^{t-1}u\left( c_t \right)}
\\
\mathrm{s}.\mathrm{t}. \begin{cases}
	A_t=\left( 1+r \right) \left( A_t+y_t-c_t \right) , t=2,\cdots ,T\\
	A_0, A_{T+1}=0\\
	A_t\in \mathbb{R}\\
	c_t\in \mathbb{R} _+\\
\end{cases}
$$

# Solver Setting

See install_ipopt.md for a tutorial on how to install the nonlinear solver ipopt.

In [1]:
import pyomo.environ as pyo

SOLVER = pyo.SolverFactory('ipopt')

# Paramater Setting

In [2]:
T = 3
beta = 1
theta = 0.5
r = 0.1

y = list(range(T)) # [0,1,2,...,T]

def u(x):
    return x ** (1 - theta) / (1 - theta)

# Create Model

In [3]:
# create model
model = pyo.ConcreteModel("Life-Cycle Model")

# create decision variables
model.c = pyo.Var(range(T), bounds=(1e-20, None))
model.A = pyo.Var(range(T + 1), bounds=(None, None))
model.obj = pyo.Objective(expr=sum(u(model.c[t]) * beta ** t for t in range(T)), sense=pyo.maximize)

# create constraint
model.cons = pyo.ConstraintList()
model.cons.add(0 == model.A[0])
model.cons.add(0 == model.A[T])
for t in range(T):
    model.cons.add((1 + r) * (model.A[t] + y[t] - model.c[t]) == model.A[t + 1])

# display updated model
model.display()

Model Life-Cycle Model

  Variables:
    c : Size=3, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 : 1e-20 :  None :  None : False :  True :  Reals
          1 : 1e-20 :  None :  None : False :  True :  Reals
          2 : 1e-20 :  None :  None : False :  True :  Reals
    A : Size=4, Index=A_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
ERROR: evaluating object as numeric value: c[0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object c[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value

In [4]:
results = SOLVER.solve(model, tee=True)

model.pprint() # display the whole model

model.obj.display()
model.c.display()
model.A.display()
print("End")

Ipopt 3.14.16: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:       11
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        7
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        5
Total number